<a href="https://colab.research.google.com/github/premsai-pendela/Cyberbullying-detection-Major-/blob/main/Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
data = pd.read_csv("/content/cyberbullying_tweets.csv")
data.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [ ]:
texts = data['tweet_text'].tolist()
labels = data['cyberbullying_type'].tolist()

In [ ]:
glove_model = api.load("glove-twitter-100")

[==================================================] 100.0% 387.1/387.1MB downloaded


In [ ]:
# Preprocess the text data and convert to numerical vectors using GloVe embeddings
embedding_size = glove_model.vector_size

In [ ]:
def text_to_vector(text):
    words = text.lower().split()
    vector = np.zeros(embedding_size)
    count = 0
    for word in words:
        if word in glove_model:
            vector += glove_model[word]
            count += 1
    if count != 0:
        vector /= count
    return vector

In [ ]:
vectorized_texts = np.array([text_to_vector(text) for text in texts])

In [ ]:
#Split the dataset into training and testing sets
train_vectors, test_vectors, train_labels, test_labels = train_test_split(
    vectorized_texts, labels, test_size=0.25, random_state=42
)

In [ ]:
#Build the neural network model
input_dim = embedding_size
num_classes = len(set(labels))

input_layer = Input(shape=(input_dim,))
x = Dense(512, activation='relu')(input_layer)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
#Fine-tuning
model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 7: Label encoding
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [ ]:
# Convert labels to one-hot encoded vectors
train_labels_encoded = to_categorical(train_labels_encoded, num_classes=num_classes)
test_labels_encoded = to_categorical(test_labels_encoded, num_classes=num_classes)

In [ ]:
#Model training
model.fit(train_vectors, train_labels_encoded, batch_size=126, epochs=200)

Epoch 1/200
284/284 [==============================] - 5s 18ms/step - loss: 0.1697 - accuracy: 0.9314
Epoch 2/200
284/284 [==============================] - 3s 11ms/step - loss: 0.1742 - accuracy: 0.9297
Epoch 3/200
284/284 [==============================] - 3s 11ms/step - loss: 0.1651 - accuracy: 0.9322
Epoch 4/200
284/284 [==============================] - 3s 10ms/step - loss: 0.1735 - accuracy: 0.9310
Epoch 5/200
284/284 [==============================] - 4s 14ms/step - loss: 0.1757 - accuracy: 0.9303
Epoch 6/200
284/284 [==============================] - 3s 11ms/step - loss: 0.1785 - accuracy: 0.9287
Epoch 7/200
284/284 [==============================] - 3s 11ms/step - loss: 0.1738 - accuracy: 0.9288
Epoch 8/200
284/284 [==============================] - 3s 11ms/step - loss: 0.1729 - accuracy: 0.9303
Epoch 9/200
284/284 [==============================] - 4s 14ms/step - loss: 0.1719 - accuracy: 0.9304
Epoch 10/200
284/284 [==============================] - 3s 10ms/step - loss: 0.169

In [ ]:
test_predictions = model.predict(test_vectors)
test_predictions = np.round(test_predictions).astype(int)

373/373 [==============================] - 1s 4ms/step


In [ ]:
target_names = label_encoder.classes_
classification_rep = classification_report(test_labels_encoded, test_predictions, target_names=target_names)
print('Classification Report:\n', classification_rep)

Classification Report:
                      precision    recall  f1-score   support

                age       0.95      0.92      0.93      1981
          ethnicity       0.92      0.88      0.90      2010
             gender       0.81      0.76      0.78      1922
  not_cyberbullying       0.48      0.49      0.49      2062
other_cyberbullying       0.48      0.47      0.47      2008
           religion       0.88      0.88      0.88      1940

          micro avg       0.75      0.73      0.74     11923
          macro avg       0.75      0.73      0.74     11923
       weighted avg       0.75      0.73      0.74     11923
        samples avg       0.73      0.73      0.73     11923



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
# Calculate accuracy, precision, recall, and F1-score
accuracy=accuracy_score(test_labels_encoded, test_predictions)
precision = precision_score(test_labels_encoded, test_predictions, average='weighted')
recall = recall_score(test_labels_encoded, test_predictions, average='weighted')
f1 = f1_score(test_labels_encoded, test_predictions, average='weighted')

# Print the calculated scores
print("Accuracy: ",accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)



Accuracy:  0.7317789147026755
Precision: 0.7484224672977989
Recall: 0.7317789147026755
F1-score: 0.7398584109777233


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
conf_matrix = multilabel_confusion_matrix(test_labels_encoded, test_predictions)
print(conf_matrix)
tn = conf_matrix[0,0]
fp = conf_matrix[0,1]
specificity = tn/(tn+fp)
print(specificity)


[[[9846   96]
  [ 163 1818]]

 [[9752  161]
  [ 239 1771]]

 [[9649  352]
  [ 465 1457]]

 [[8761 1100]
  [1043 1019]]

 [[8878 1037]
  [1064  944]]

 [[9744  239]
  [ 224 1716]]]
[0.98371466 0.05015674]
